###Libraries

In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

### METHOD 1: Using Pandas  Dataframe

In [2]:
# METHOD 1: convert csv file to pandas dataframe
colnames=['bounding_box','city','country','state', 'text','tid','timestamp','uid']
pandas_df = pd.read_csv('./newtweets_20151207_1228.csv', delimiter=',', skiprows=1, names=colnames, usecols=[1,2,3,4,5,6,7,8])
pandas_df.head(5)

bounding_box          city        country       state                                               text                 tid      timestamp         uid
0  {u'type': u'Polygon', u'coordinates': [[[-63.3...  Buenos Aires      Argentina   Argentina       "Si empesamos hablar se pudre con la doli.."  673735583940878338  1449466055270  2564256692
1  {u'type': u'Polygon', u'coordinates': [[[-45.1...      Cruzeiro         Brasil   São Paulo  "que paia stalkear a pessoa e curtir a foto se...  673735583693451264  1449466055211  2237980915
2  {u'type': u'Polygon', u'coordinates': [[[-73.9...      Fort Lee  United States          NJ                                           "Hmuuuu"  673735584129671168  1449466055315   334470631
3  {u'type': u'Polygon', u'coordinates': [[[-80.2...    Boca Raton  United States          FL  "Se respira el aire de cambio. #elcambiovieney...  673735584096063488  1449466055307    69791277
4  {u'type': u'Polygon', u'coordinates': [[[-73.3...     Venezuela      Venezuela   Venezuela  "Y va a caer, y va a caer.. este gobierno va a...  673735584549101568  1449466055415   201410531

In [3]:
pandas_df.country.head()

0        Argentina
1           Brasil
2    United States
3    United States
4        Venezuela
Name: country, dtype: object

In [4]:
usdf = pandas_df[pandas_df.country == "United States"]
usdf.head(5)

bounding_box          city        country state                                               text                 tid      timestamp         uid
2  {u'type': u'Polygon', u'coordinates': [[[-73.9...      Fort Lee  United States    NJ                                           "Hmuuuu"  673735584129671168  1449466055315   334470631
3  {u'type': u'Polygon', u'coordinates': [[[-80.2...    Boca Raton  United States    FL  "Se respira el aire de cambio. #elcambiovieney...  673735584096063488  1449466055307    69791277
5  {u'type': u'Polygon', u'coordinates': [[[-119....  Farmersville  United States    CA                  "@fcornejo9 they're this week???"  673735584238559233  1449466055341   572075963
7  {u'type': u'Polygon', u'coordinates': [[[-85.6...       Georgia  United States   USA  "Squad goals with bae. 😊 https://t.co/TBKgqhQ...  673735584142114822  1449466055318  1602156134
9  {u'type': u'Polygon', u'coordinates': [[[-84.3...      Richmond  United States    KY  "my girl I miss you so much and all of the tac...  673735584880459777  1449466055494    50910424

### Extract address info from gps coordinates

In [5]:
from geopy.geocoders import Nominatim
geolocator = Nominatim()
import ast

In [6]:
#test
latitude = 47.495315
longitude = -122.436232
tweetloc = geolocator.reverse((latitude, longitude))
tweetaddr = (tweetloc.raw)
print tweetaddr


{u'display_name': u'Dolphin Point Trail Southwest, Vashon, King County, Washington, 98070, United States of America', u'place_id': u'47783845', u'lon': u'-122.4558659', u'osm_type': u'way', u'licence': u'Data \xa9 OpenStreetMap contributors, ODbL 1.0. http://www.openstreetmap.org/copyright', u'osm_id': u'6432047', u'lat': u'47.501025', u'address': {u'footway': u'Dolphin Point Trail Southwest', u'locality': u'Vashon', u'country': u'United States of America', u'county': u'King County', u'state': u'Washington', u'postcode': u'98070', u'country_code': u'us'}}


In [7]:
def find_tweet_address(gps_polygon_text):
    location_dict = None
    gps_polygon_dict = ast.literal_eval(gps_polygon_text)
    longitude =  gps_polygon_dict['coordinates'][0][0][0]
    latitude =  gps_polygon_dict['coordinates'][0][0][1]
    tweetlocation = geolocator.reverse((latitude, longitude))
    tweetaddress_fields = (tweetlocation.raw)
    try:
        county = tweetaddress_fields['address']['county']
        state = tweetaddress_fields['address']['state']
        zipcode = tweetaddress_fields['address']['postcode']
    except:
        county = ''
        state = ''
        zipcode = ''      
    location_dict = dict(county=county, state=state, zipcode=zipcode)
    return location_dict['zipcode']

In [8]:
%%time 
usdf = pandas_df[pandas_df.country == "United States"].head(50)
usdf['zipcode'] = usdf.bounding_box.apply(lambda x: find_tweet_address(x))  

CPU times: user 655 ms, sys: 37.9 ms, total: 693 ms
Wall time: 22 s


In [9]:
usdf.head(5)

bounding_box          city        country state                                               text                 tid      timestamp         uid zipcode
2  {u'type': u'Polygon', u'coordinates': [[[-73.9...      Fort Lee  United States    NJ                                           "Hmuuuu"  673735584129671168  1449466055315   334470631   07010
3  {u'type': u'Polygon', u'coordinates': [[[-80.2...    Boca Raton  United States    FL  "Se respira el aire de cambio. #elcambiovieney...  673735584096063488  1449466055307    69791277   33076
5  {u'type': u'Polygon', u'coordinates': [[[-119....  Farmersville  United States    CA                  "@fcornejo9 they're this week???"  673735584238559233  1449466055341   572075963   93292
7  {u'type': u'Polygon', u'coordinates': [[[-85.6...       Georgia  United States   USA  "Squad goals with bae. 😊 https://t.co/TBKgqhQ...  673735584142114822  1449466055318  1602156134   32409
9  {u'type': u'Polygon', u'coordinates': [[[-84.3...      Richmond  United States    KY  "my girl I miss you so much and all of the tac...  673735584880459777  1449466055494    50910424   40475

##METHOD 2: Using Spark

In [10]:
#create spark dataframe
import findspark
findspark.init()
print findspark.find()
import pyspark
conf = (pyspark.SparkConf()
    .setMaster('local')
    .setAppName('pyspark')
    .set("spark.executor.memory", "4g"))
sc = pyspark.SparkContext(conf=conf)

/usr/local/opt/apache-spark/libexec


In [11]:
import sys
rdd = sc.parallelize(xrange(10),10)
rdd.map(lambda x: sys.version).collect()

['2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x8

In [12]:
# read csv to spark RDD then creat spark dataframe from it
alltweets_rdd = sc.textFile('./tweets.csv')
alltweets_rdd = alltweets_rdd.map(lambda line: line.split(","))
print "Total number of tweets: ", alltweets_rdd.count()
alltweets_rdd.take(2)

Total number of tweets:  37519


[[u'0',
  u'"{u\'type\': u\'Polygon\'',
  u" u'coordinates': [[[-118.082615",
  u' 33.628991]',
  u' [-118.082615',
  u' 33.756093]',
  u' [-117.91485',
  u' 33.756093]',
  u' [-117.91485',
  u' 33.628991]]]}"',
  u'Huntington Beach',
  u'United States',
  u'In need',
  u' of back rub',
  u'669375138903666692',
  u'1448426444160',
  u'1611125173'],
 [u'1',
  u'"{u\'type\': u\'Polygon\'',
  u" u'coordinates': [[[-82.87387",
  u' 38.688052]',
  u' [-82.87387',
  u' 38.76256]',
  u' [-82.811927',
  u' 38.76256]',
  u' [-82.811927',
  u' 38.688052]]]}"',
  u'Wheelersburg',
  u'United States',
  u"And I'm always tired but never of you",
  u'669375138958307328',
  u'1448426444173',
  u'3165522051']]

In [13]:
from pyspark.sql import SQLContext, Row
from pyspark.sql.types import *
sqlsc=SQLContext(sc)

In [15]:
#create spark dataframe from rdd of csv file
alltweets_rdd = sc.textFile('./tweets.csv')
alltweets_rdd = alltweets_rdd.map(lambda line: line.split(","))
header = alltweets_rdd.first()
#alltweets_rdd = alltweets_rdd.filter(lambda line:line != header)
#alltweets_df0 = alltweets_rdd.map(lambda line: Row(bounding_box=line[1:10], city=line[10], country=line[11], 
#                              text=line[12:-3], tid=line[-3], timestamp=line[-2], uid=line[-1])).toDF()
alltweets_df0 = alltweets_rdd.map(lambda line: Row(bounding_box=line[1:10], city=line[10], country=line[11], 
                              text=line[12:-3], tid=line[-3], timestamp=line[-2], uid=line[-1]))
alltweets_df = sqlsc.createDataFrame(alltweets_df0)
#alltweets_df.registerTempTable("alltweets_df")
alltweets_df.printSchema()

print type(alltweets_rdd)
print alltweets_rdd.count()
print type(alltweets_df)

root
 |-- bounding_box: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- text: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- tid: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- uid: string (nullable = true)

<class 'pyspark.rdd.PipelinedRDD'>
37519
<class 'pyspark.sql.dataframe.DataFrame'>


In [16]:
# RDD actions like count() or take() do not seem to work 
# for the alltweets_df object. Is it because dataset is too big?
# Not sure why the following return an IndexError: "list index out of range"  
#alltweets_df.count()
#alltweets_df.take(2)
#alltweets_df.show(5)

In [17]:
#alltweets_df.head()
alltweets_df.first()

Row(bounding_box=[u'"{u\'type\': u\'Polygon\'', u" u'coordinates': [[[-118.082615", u' 33.628991]', u' [-118.082615', u' 33.756093]', u' [-117.91485', u' 33.756093]', u' [-117.91485', u' 33.628991]]]}"'], city=u'Huntington Beach', country=u'United States', text=[u'In need', u' of back rub'], tid=u'669375138903666692', timestamp=u'1448426444160', uid=u'1611125173')

In [18]:
alltweets_df.select("text").first()

Row(text=[u'In need', u' of back rub'])

In [19]:
tweet_states = alltweets_df.map(lambda r: find_tweet_address_v2(r.bounding_box))

In [20]:
#tweet_states.take(3)